In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import sys
sys.path.append('/content/gdrive/My Drive')

In [ ]:
pip install transformers

Importation des fonctions utiles

In [ ]:
import functions

Importation de la base de données extraite de Doccano

In [ ]:
from google.colab import files
uploaded = files.upload()

In [11]:
import io
import pandas as pd
df = pd.read_json(io.BytesIO(uploaded['file (1).json']), lines = True)

On sépare les données nettoyées en deux.

In [ ]:
train_dataset, val_dataset = functions.get_train_val(df)

In [13]:
import torch 

if torch.cuda.is_available():      
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
from transformers import BertForTokenClassification, AdamW, BertConfig
model = BertForTokenClassification.from_pretrained("bert-base-uncased",
    num_labels = 2, 
    output_attentions = False, 
    output_hidden_states = False
)

model.cuda()


In [15]:
def dummy_data_collector(features):
    batch = {}
    batch['input_ids'] = torch.stack([f[0] for f in features])
    batch['attention_mask'] = torch.stack([f[1] for f in features])
    batch['labels'] = torch.stack([f[2] for f in features])
    
    return batch

In [16]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [25]:
torch.cuda.empty_cache()

In [29]:
from transformers import Trainer, TrainingArguments

EPOCHS = 16  #à augmenter pour des meilleurs résultats
BATCH_SIZE = 8 #à augmenter jusqu'à pb de mémoire

LR = 2e-5 #learning rate
EPS = 1e-8 #epsilon 

training_args = TrainingArguments(
    output_dir='./results',         
    num_train_epochs=EPOCHS,             
    per_device_train_batch_size=BATCH_SIZE,  
    per_device_eval_batch_size=16,   
    warmup_steps=500,                
    weight_decay=0.01,  
    learning_rate = LR,
    adam_epsilon = EPS,             
    logging_dir='./logs',            
)

trainer = Trainer(
    model=model,
    data_collator = dummy_data_collector,
    compute_metrics = compute_metrics,
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset            
)

In [ ]:
trainer.train()

Step,Training Loss


In [ ]:
trainer.evaluate()